<a href="https://colab.research.google.com/github/gdeotale/E4P2/blob/master/Assignment9/Neural_word_embeddings_and_sentiment_analysis_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Manage Imports

In [6]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext import datasets
import random
import spacy

nlp = spacy.load('en')
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

#Download IMDB dataset and make train/test/val split

In [7]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 180k/84.1M [00:00<00:53, 1.56MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 60.0MB/s]


In [8]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

#Mostly to annotate data to numbers

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 399202/400000 [00:16<00:00, 25247.40it/s]

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 204663), (',', 194438), ('.', 166574), ('a', 110568), ('and', 110202), ('of', 101583), ('to', 94718), ('is', 76565), ('in', 61968), ('I', 54693), ('it', 54022), ('that', 49613), ('"', 44620), ("'s", 43656), ('this', 42490), ('-', 37460), ('/><br', 36055), ('was', 35552), ('as', 30642), ('with', 30203)]


#Integer to string from annotation

In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


#String to integer from annotation

In [ ]:
print(LABEL.vocab.stoi)

print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fbff2094d90>, {'neg': 0, 'pos': 1})
defaultdict(<function _default_unk_index at 0x7fbff2094d90>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'a': 5, 'and': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, '(': 27, ')': 28, 'on': 29, "n't": 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'one': 37, 'he': 38, 'at': 39, '!': 40, 'by': 41, 'all': 42, 'an': 43, 'who': 44, 'they': 45, 'like': 46, 'from': 47, 'so': 48, 'or': 49, 'her': 50, 'about': 51, "'": 52, 'has': 53, 'out': 54, 'just': 55, 'It': 56, 'do': 57, '?': 58, 'some': 59, 'good': 60, 'more': 61, 'would': 62, 'very': 63, 'up': 64, 'what': 65, 'This': 66, 'there': 67, 'time': 68, 'can': 69, 'which': 70, 'when': 71, 'if': 72, 'had': 73, 'she': 74, 'only': 75, 'story': 76, 'were': 77, 'see': 78, 

#Create Iterator for training

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

#Model

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        #embedded = [sent len, batch size, emb dim]
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
# ! pip install torchsummaryX
# from torchsummaryX import summary
# inputs1 = torch.zeros((512,1), dtype = torch.long) #[ sentence length , batch size]
# inputs2 = torch.tensor(1, dtype = torch.long)
# print(inputs1)
# print(inputs2)
# inputs = (inputs1, inputs2)
# summary(model, inputs)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4597,  0.5048,  0.7880,  ..., -0.5276,  1.6481,  1.7779],
        [ 1.4982,  0.5545,  1.1260,  ...,  0.8116, -1.2200,  0.3242],
        [ 0.3485,  0.2133, -0.4966,  ..., -0.3339, -0.9237, -0.4172]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4597,  0.5048,  0.7880,  ..., -0.5276,  1.6481,  1.7779],
        [ 1.4982,  0.5545,  1.1260,  ...,  0.8116, -1.2200,  0.3242],
        [ 0.3485,  0.2133, -0.4966,  ..., -0.3339, -0.9237, -0.4172]])


#Loss, optimizer, accuracy for training

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        # print(text.shape, text_lengths.shape)
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#Train and save model

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.659 | Train Acc: 59.82%
	 Val. Loss: 0.670 |  Val. Acc: 59.39%
Epoch: 02 | Epoch Time: 0m 39s
	Train Loss: 0.642 | Train Acc: 62.66%
	 Val. Loss: 0.495 |  Val. Acc: 77.43%
Epoch: 03 | Epoch Time: 0m 41s
	Train Loss: 0.538 | Train Acc: 73.66%
	 Val. Loss: 0.531 |  Val. Acc: 75.86%
Epoch: 04 | Epoch Time: 0m 41s
	Train Loss: 0.463 | Train Acc: 77.92%
	 Val. Loss: 0.364 |  Val. Acc: 84.66%
Epoch: 05 | Epoch Time: 0m 41s
	Train Loss: 0.316 | Train Acc: 86.94%
	 Val. Loss: 0.323 |  Val. Acc: 86.51%


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.363 | Test Acc: 84.80%


In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is not good")

0.9288532137870789

In [ ]:
predict_sentiment(model, "i must say the film is too bad")

0.03673931583762169

In [ ]:
predict_sentiment(model, "This film is great")

0.9938331842422485

In [ ]:
predict_sentiment(model, "This film is pathetic")

0.01814287155866623

#Serializing model

In [ ]:
scripted_model = torch.jit.script(model.to('cpu'))
torch.jit.save(scripted_model, 'sentimental_analysis_jit.pt')

#Loading model code for deployment

In [22]:
import pickle
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(dict1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
import spacy
import torch
from torchtext import data
import pickle
with open('dictionary.pickle', 'rb') as handle:
    dict1 = pickle.load(handle)
nlp = spacy.load('en')

In [5]:
loaded_compiled_model = torch.jit.load('sentimental_analysis_jit.pt')

In [6]:
sentence = "i must say the film is too bad"
tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
indexed = [dict1[t] for t in tokenized]
length = [len(indexed)]
tensor = torch.LongTensor(indexed).to('cpu')
tensor = tensor.unsqueeze(1)
length_tensor = torch.LongTensor(length)
prediction = torch.sigmoid(loaded_compiled_model(tensor, length_tensor)).item()

In [7]:
print(prediction)

0.036739323288202286
